# Fallback models

In many cases it is good to have a fallback model. There are solution like [LiteLLM](), an LLM gateway that allows you to configure accesses, fallback models, quotas and much more.

If OpenAI API key is already exported, you can install `litellm[proxy]` and invoke, for example

```
litellm --model gpt-3.5-turbo
```

You will have it running immediatelly for testing the connection. Then, you can simply invoke the library and connecto your agent to the endpoint that redirects the traffic.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(
    openai_api_base="http://0.0.0.0:4000", # set openai_api_base to the LiteLLM Proxy
    model = "gpt-3.5-turbo",
    temperature=0.1
)

messages = [
    SystemMessage(
        content="You are a helpful assistant that im using to make a test request to."
    ),
    HumanMessage(
        content="test from litellm. tell me why it's amazing in 1 sentence"
    ),
]
response = chat(messages)

print(response)

/tmp/ipykernel_84066/2642415190.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(
/tmp/ipykernel_84066/2642415190.py:18: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


AuthenticationError: Error code: 401 - {'error': {'message': 'litellm.AuthenticationError: AuthenticationError: OpenAIException - Incorrect API key provided: sk-proj-********************************************************************************************************************************************************j5oA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': None, 'param': None, 'code': '401'}}

### Locally

You can also try with locally deployed models. Using solutions like Ollama or [vLLM](https://docs.vllm.ai/en/stable/index.html) you can easily host and serve you own models. Although, machine specifications may be higher than the usual laptop, going for higher RAM and a little bit of GPU if possible.

Feel free to check it out.

In [2]:
from langchain_community.llms import VLLM

llm = VLLM(
    model="TheBloke/Llama-2-7b-Chat-AWQ",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    vllm_kwargs={"quantization": "awq"}, # quantized
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

print(llm.invoke("What is the capital of France ?"))

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

INFO 07-31 08:18:21 [config.py:1604] Using max model len 4096
INFO 07-31 08:18:22 [awq_marlin.py:120] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
WARNING 07-31 08:18:22 [config.py:1084] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-31 08:18:22 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:17<00:00,  3.85it/s]


INFO 07-31 08:21:33 [gpu_model_runner.py:2485] Graph capturing finished in 18 secs, took 0.80 GiB
INFO 07-31 08:21:34 [core.py:193] init engine (profile, create kv cache, warmup model) took 58.83 seconds


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


The capital of France is Paris. Paris is the largest city in France and is located in the northern central part of the country. It is a major cultural, economic, and political center in Europe and is known for its iconic landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum.
The capital of France has a long and complex history, with the city being part of various kingdoms, empires, and republics over the centuries. The modern city of Paris was founded in the 3rd century BC by the Gaul tribe of the Parisii, who named the
